# Bruno R. Batista - Projeto 3: Algoritmos de Aprendizado de Máquina

Base de dados analisada: Breast Cancer (#9)

Arquivo anterior (tratamento de dados): prj02.py

In [1]:
#pip install scikit-learn

In [2]:
#pip install numpy==1.24.0

In [3]:
# Puxar as informações tratadas do projeto anterior
import prj02
import random

tabTreino = prj02.novoTreino
print(len(tabTreino))
tabTeste = prj02.tabTeste
tiposEntr = prj02.tiposEntr

# Transformar os valores booleanos em 0s e 1s
# O sklearn só funciona com inteiros
for col in range(0,len(tabTreino)):
    if type(tabTreino[col][0]) != type(1):
        for linha in range(0,len(tabTreino[col])):
            valor = tabTreino[col][linha]
            if type(valor) == bool:
                if valor: tabTreino[col][linha] = 1
                else: tabTreino[col][linha] = 0
            elif type(valor) == str: 
                tabTreino[col][linha] = int(valor)

Dados carregados

Tipos de entrada possíveis por coluna:
[['no-recurrence-events', 'recurrence-events'], ['10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99'], ['lt40', 'ge40', 'premeno'], ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59'], ['0-2', '3-5', '6-8', '9-11', '12-14', '15-17', '18-20', '21-23', '24-26', '27-29', '30-32', '33-35', '36-39'], ['yes', 'no'], ['1', '2', '3'], ['left', 'right'], ['left_up', 'left_low', 'right_up', 'right_low', 'central'], ['no', 'yes']]

Tipos de cada atributos:
{'Class': 'qualitativo nominal', 'age': 'qualitativo intervalar', 'menopause': 'qualitativo nominal', 'tumor-size': 'qualitativo intervalar', 'inv-nodes': 'qualitativo intervalar', 'node-caps': 'qualitativo nominal', 'deg-malig': 'qualitativo ordinal', 'breast': 'qualitativo nominal', 'breast-quad': 'qualitativo nominal', 'irradiat': 'qualitativo nominal'}
Atributo 'Class':
	Moda - no-recurrence-events
	Frequ

# Parte 1: Definir da técnica de validação a ser utilizada (cross-validation, hold-out, leave-one-out, etc)

Inicialmente usa hold-out (dividir a base de treino em 2/3).

Futuramente usa leave-one-out (rodar com cada linha como a base de validação), pois a base de treino já é pequena (cerca de 191 linhas).

Como o modelo da Árvore de Decisão não possui parametros de entrada, só foi feito o ajuste dos outros 2 modelos.

In [4]:
# Parte 1.1: criar o algoritmo das separações de validação

treinoHoldOut = []
avaliacaoHoldOut = []

# Inicialmente é o método Hold-Out
# Já foi feita uma função de separação de grupos no trabalho anterior, mas ela teve que ser adaptada para esse trabalho
def separarTabelas(tabIni, nLinhas):
    treino = []
    for col in tabIni: treino.append(col.copy())
        
    teste = []
    for col in tabIni: teste.append([])

    tam = len(tabIni[0])/3
    for i in range(0,int(tam)):
        # A escolha dos testes é aleatória
        rand = random.randrange(0,len(treino[1]))
        for j in range(0,len(tabIni)):
            teste[j].insert(0,treino[j].pop(rand))
    return [treino, teste]
tabs = separarTabelas(tabTreino,5)
treinoHoldOut = tabs[0]
avaliacaoHoldOut = tabs[1]

In [5]:
# Adequar as bases de dados ao sklearn
setY_treino = treinoHoldOut.pop(0)
setX_treino = list(zip(treinoHoldOut[0],treinoHoldOut[1],treinoHoldOut[2],treinoHoldOut[3],
                       treinoHoldOut[4],treinoHoldOut[5],treinoHoldOut[6],treinoHoldOut[7],
                       treinoHoldOut[8],treinoHoldOut[9],treinoHoldOut[10],treinoHoldOut[11],
                       treinoHoldOut[12],treinoHoldOut[13],treinoHoldOut[14]))

In [6]:
# Parte 2.1: fazer as funções de cálculo de métrica
# Todos os cálculos estão na mesma função

# Essa função auxilia o leave-one-out, então teve que ser feita antes

def metricas(real,predito):
    mConf = {'VP': 0, 'VN':0, 'FP':0, 'FN':0}
    for i in range(0,len(real)):
        if (real[i] == 1 and predito[i] == 1): mConf['VP'] = mConf['VP'] +1
        elif (real[i] == 0 and predito[i] == 1): mConf['FP'] = mConf['FP'] +1
        elif (real[i] == 1 and predito[i] == 0): mConf['FN'] = mConf['FN'] +1
        else: mConf['VN'] = mConf['VN'] +1
            
    VP = mConf["VP"]
    VN = mConf["VN"]
    FP = mConf["FP"]
    FN = mConf["FN"]   
    
    # Calcular o recall
    if (VP + FN) != 0: recall = VP / (VP + FN)
    else: recall = 99999
    
    # Calcular a precisão
    if (VN + FP) != 0: precisao = VN / (VN + FP)
    else: precisao = 99999
    
    return {"matConf":mConf, "recall":recall, "precisao":precisao}

In [7]:
# Depois, foi aplicado o método Leave-one-out, mais condizente com a base de dados
# Como as métricas são médias nesse caso
'''from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

def leave_one_out(original):
    mod_KNN = KNeighborsClassifier(n_neighbors=4)
    mod_AdD = DecisionTreeClassifier()
    mod_MLP = MLPClassifier(max_iter=300,learning_rate_init=0.01,hidden_layer_sizes=(100,),early_stopping=True)

    met_KNN = {"recall":0, "precisao":0}
    met_AdD = {"recall":0, "precisao":0}
    met_MLP = {"recall":0, "precisao":0}
    
    for y in range(0,len(original[0])):
        tab = []
        for col in original: tab.append(col.copy())
            
        setY_treino = tab.pop(0)
        setX_treino = list(zip(tab[0],tab[1],tab[2],tab[3],tab[4],tab[5],tab[6],tab[7],
                       tab[8],tab[9],tab[10],tab[11],tab[12],tab[13],tab[14]))

        setY_validacao = [setY_treino.pop(y)]
        setX_validacao = [setX_treino.pop(y)]

        mod_KNN.fit(setX_treino,setY_treino)
        p_KNN = mod_KNN.predict(setX_validacao)

        mod_AdD.fit(setX_treino,setY_treino)
        p_AdD = mod_AdD.predict(setX_validacao)

        mod_MLP.fit(setX_treino,setY_treino)
        p_MLP = mod_KNN.predict(setX_validacao)

        m_KNN = metricas(real=setY_validacao,predito=p_KNN)
        m_AdD = metricas(real=setY_validacao,predito=p_AdD)
        m_MLP = metricas(real=setY_validacao,predito=p_MLP)

        met_KNN["recall"] = met_KNN["recall"]+m_KNN["recall"]
        met_KNN["precisao"] = met_KNN["precisao"]+m_KNN["precisao"]
        
        met_AdD["recall"] = met_AdD["recall"]+m_AdD["recall"]
        met_AdD["precisao"] = met_AdD["precisao"]+m_AdD["precisao"]
        
        met_MLP["recall"] = met_MLP["recall"]+m_MLP["recall"]
        met_MLP["precisao"] = met_MLP["precisao"]+m_MLP["precisao"]

    tam = len(original[0])
    met_KNN["recall"] = met_KNN["recall"] / tam
    met_KNN["precisao"] = met_KNN["precisao"] / tam
    
    met_AdD["recall"] = met_AdD["recall"] / tam
    met_AdD["precisao"] = met_AdD["precisao"] / tam
    
    met_MLP["recall"] = met_MLP["recall"] / tam
    met_MLP["precisao"] = met_MLP["precisao"] / tam

    return {'KNN':met_KNN, 'AdD':met_AdD, 'MLP':met_MLP}
    
LON = leave_one_out(tabTreino)
print("Resultados do K-NN:\n",LON['KNN'])
print("\nResultados do Árvore de Decisão:\n",LON['AdD'])
print("\nResultados do MLP:\n",LON['MLP'])'''

'from sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.neural_network import MLPClassifier\n\ndef leave_one_out(original):\n    mod_KNN = KNeighborsClassifier(n_neighbors=4)\n    mod_AdD = DecisionTreeClassifier()\n    mod_MLP = MLPClassifier(max_iter=300,learning_rate_init=0.01,hidden_layer_sizes=(100,),early_stopping=True)\n\n    met_KNN = {"recall":0, "precisao":0}\n    met_AdD = {"recall":0, "precisao":0}\n    met_MLP = {"recall":0, "precisao":0}\n    \n    for y in range(0,len(original[0])):\n        tab = []\n        for col in original: tab.append(col.copy())\n            \n        setY_treino = tab.pop(0)\n        setX_treino = list(zip(tab[0],tab[1],tab[2],tab[3],tab[4],tab[5],tab[6],tab[7],\n                       tab[8],tab[9],tab[10],tab[11],tab[12],tab[13],tab[14]))\n\n        setY_validacao = [setY_treino.pop(y)]\n        setX_validacao = [setX_treino.pop(y)]\n\n        mod_KNN.fit(setX_treino,setY_treino)\n     

# Parte 2: Definir as métricas para a avaliação do modelo (acurácia, precisão, recall, matriz de confusão, etc)

Devido ao tamanho pequeno da base, já foi definido que a acurácia não é muito relevante para esta análise, e que a precisão e o recall devem ser mais visados nesse caso.

Fórmulas:
- Recall (sensibilidade): VP / (VP + FN)
- Precisão (Especificidade): VN / (VN + FP)

A função das métricas já foi posta no item anterior

# Parte 3: Fazer o algoritmo de Baseline

O algoritmo de Baseline simplesmente vê qual é a classe majoritária e coloca todas as predições com essa classe.

In [8]:
# Parte 3.1: Adaptar a base de avaliação para os modelos do sklearn

# Preparar o conjunto de avaliação do modelo
setY_avaliacao = avaliacaoHoldOut.pop(0)
setX_avaliacao = list(zip(avaliacaoHoldOut[0],avaliacaoHoldOut[1],avaliacaoHoldOut[2],avaliacaoHoldOut[3],
                       avaliacaoHoldOut[4],avaliacaoHoldOut[5],avaliacaoHoldOut[6],avaliacaoHoldOut[7],
                       avaliacaoHoldOut[8],avaliacaoHoldOut[9],avaliacaoHoldOut[10],avaliacaoHoldOut[11],
                       avaliacaoHoldOut[12],avaliacaoHoldOut[13],avaliacaoHoldOut[14]))

In [9]:
# Parte 3.2: fazer o algoritmo de baseline

def predicaoBaseline(setY,tam):
    predicao = []
    
    freqs = prj02.frequencia(setY)
    if freqs[0] > freqs[1]: valor = 0
    else: valor = 1
    
    for i in range(0,tam): predicao.append(valor)
    return predicao

# Teste com a o set de validação no baseline
baseline = predicaoBaseline(setY_treino,len(setY_avaliacao))   
print(metricas(baseline,setY_avaliacao))

{'matConf': {'VP': 0, 'VN': 43, 'FP': 17, 'FN': 0}, 'recall': 99999, 'precisao': 0.7166666666666667}


# Parte 3.5: preparar a tabela de dados "reais" para ser usada nos modelos

A tabela de teste separada no meio do projeto 2 tem todos os dados originais, em strings. Eles devem virar sets com inteiros para os modelos funcionarem.

In [10]:
# Parte 4.1: criar os sets para os modelos de verdade

# Preparar o conjunto de treino completo do modelo
setY_modeloFinal = tabTreino.pop(0)
setX_modeloFinal = list(zip(tabTreino[0],tabTreino[1],tabTreino[2],tabTreino[3],
                       tabTreino[4],tabTreino[5],tabTreino[6],tabTreino[7],
                       tabTreino[8],tabTreino[9],tabTreino[10],tabTreino[11],
                       tabTreino[12],tabTreino[13],tabTreino[14]))

In [11]:
# Preparar o conjunto de dados "reais"
# Processo retirado do trabalho anterior
tabReal = []
for col in tabTreino: tabReal.append([])
tabReal.append([]) # tabTreino perdeu uma coluna ao fazer o set
    
# 1o) Converter os sim's e não's
classLine = []
nodeLine = []
breastLine = []
irradiatLine = []
for y in range(0,len(tabTeste[0])):
    if tabTeste[0][y] == 'recurrence-events': classLine.append(1)
    else: classLine.append(0)
        
    if tabTeste[5][y] == 'yes': nodeLine.append(1)
    else: nodeLine.append(0)
        
    if tabTeste[7][y] == 'right': breastLine.append(1)
    else: breastLine.append(0)
        
    if tabTeste[9][y] == 'yes': irradiatLine.append(1)
    else: irradiatLine.append(0)
        
tabReal[0] = classLine
tabReal[7] = nodeLine
tabReal[9] = breastLine
tabReal[15] = irradiatLine

In [12]:
# 2o) Converter os intervalares em ordinais
ageLine = []
tumorLine = []
invLine = []
degLine = []
for y in range(0,len(tabTeste[0])):
    newAge = prj02.tiposEntrOriginal[1].index(tabTeste[1][y])
    ageLine.append(newAge)
    
    newTumor = prj02.tiposEntrOriginal[3].index(tabTeste[3][y])
    tumorLine.append(newTumor)
    
    newInv = prj02.tiposEntrOriginal[4].index(tabTeste[4][y])
    invLine.append(newInv)
    
    newDeg = prj02.tiposEntrOriginal[4].index(tabTeste[4][y])
    degLine.append(newInv)
    
tabReal[1] = ageLine
tabReal[5] = tumorLine
tabReal[6] = invLine
tabReal[8] = degLine

In [13]:
# 3o) Converter os categóricos em colunas
# Fazer as colunas novas

# Preencher do novo jeito
for y in range(0,len(tabTeste[0])):
    # Dados de 'menopause'
    meno = tabTeste[2][y]
    if meno == 'lt40': tabReal[2].append(1)
    else: tabReal[2].append(0)
    if meno == 'ge40': tabReal[3].append(1)
    else: tabReal[3].append(0)
    if meno == 'premeno': tabReal[4].append(1)
    else: tabReal[4].append(0)
    
    quad = tabTeste[8][y]
    if quad == 'left-up': tabReal[10].append(1)
    else: tabReal[10].append(0)
    if quad == 'left-low': tabReal[11].append(1)
    else: tabReal[11].append(0)
    if quad == 'right-up': tabReal[12].append(1)
    else: tabReal[12].append(0)
    if quad == 'right-low': tabReal[13].append(1)
    else: tabReal[13].append(0)
    if quad == 'central': tabReal[14].append(1)
    else: tabReal[14].append(0)

In [14]:
# 4o) Transformar nos sets X e Y
setY_real = tabReal.pop(0)
setX_real = list(zip(tabReal[0],tabReal[1],tabReal[2],tabReal[3],
                       tabReal[4],tabReal[5],tabReal[6],tabReal[7],
                       tabReal[8],tabReal[9],tabReal[10],tabReal[11],
                       tabReal[12],tabReal[13],tabReal[14]))

# Parte 4: predição dos dados reais com o K-NN

Usar o modelo de Classificação do K-NN com a base de Teste, e medir as suas métricas.

In [15]:
# Parte 4.1: puxar o modelo do sklearn usar com a base de treino toda
from sklearn.neighbors import KNeighborsClassifier

modelo_KNN = KNeighborsClassifier(n_neighbors=4)
modelo_KNN.fit(setX_modeloFinal,setY_modeloFinal)

KNeighborsClassifier(n_neighbors=4)

In [16]:
# Parte 4.2: testar a predição nos dados reais
predicao_KNN = modelo_KNN.predict(setX_real)
print(predicao_KNN,'\n',metricas(setY_real,predicao_KNN))

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0] 
 {'matConf': {'VP': 5, 'VN': 48, 'FP': 4, 'FN': 38}, 'recall': 0.11627906976744186, 'precisao': 0.9230769230769231}


# Parte 5: predição dos dados reais com a Árvore de Decisão

Usar o modelo de Classificação da Árvore de Decisão com a base de Teste, e medir as suas métricas.

In [17]:
# Parte 5.1: puxar o modelo do sklearn usar com a base de treino toda
from sklearn.tree import DecisionTreeClassifier

modelo_AdD = DecisionTreeClassifier()
modelo_AdD.fit(setX_modeloFinal,setY_modeloFinal)

DecisionTreeClassifier()

In [18]:
# Parte 5.2: testar a predição nos dados reais
predicao_AdD = modelo_KNN.predict(setX_real)
print(predicao_AdD,'\n',metricas(setY_real,predicao_AdD))

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0] 
 {'matConf': {'VP': 5, 'VN': 48, 'FP': 4, 'FN': 38}, 'recall': 0.11627906976744186, 'precisao': 0.9230769230769231}


# Parte 6: predição dos dados reais com o MLP

Usar o modelo de Classificação do MLP com a base de Teste, e medir as suas métricas.

In [19]:
# Parte 6.1: puxar o modelo do sklearn usar com a base de treino toda
from sklearn.neural_network import MLPClassifier

modelo_MLP = MLPClassifier(solver='lbfgs', alpha=1e-5, max_iter=300,
                    hidden_layer_sizes=(15,), random_state=1)
modelo_MLP.fit(setX_modeloFinal,setY_modeloFinal)

/home/brunobap/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,), max_iter=300,
              random_state=1, solver='lbfgs')

In [20]:
# Parte 6.2: testar a predição nos dados reais
predicao_MLP = modelo_MLP.predict(setX_real)
print(predicao_MLP,'\n')

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 1
 1 0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 0 0 0 0 0] 



# Parte 7: Análise dos resultados do algoritmo do Baseline

In [21]:
predicao_base = predicaoBaseline(setY_modeloFinal,len(setY_real))
metricas_base = metricas(setY_real,predicao_base)
print(metricas_base)

{'matConf': {'VP': 0, 'VN': 52, 'FP': 0, 'FN': 43}, 'recall': 0.0, 'precisao': 1.0}


É possível ver que a classe majoritária do conjunto de treino final foi a negativa, pois todos os itens preditos pelo algoritmo foram VN ou FN.

Com isso, o recall ficou nulo, pois nenhum item foi marcado como positivo. E a precisão foi de 1 pois não houve ocorrência de FPs.

# Parte 8: Análise dos resultados dos algoritmos do sklearn

In [22]:
print("Resultados do K-NN:\n",metricas(setY_real,predicao_KNN))
print("\nResultados do Árvore de Decisão:\n",metricas(setY_real,predicao_AdD))
print("\nResultados do MLP:\n",metricas(setY_real,predicao_MLP))

Resultados do K-NN:
 {'matConf': {'VP': 5, 'VN': 48, 'FP': 4, 'FN': 38}, 'recall': 0.11627906976744186, 'precisao': 0.9230769230769231}

Resultados do Árvore de Decisão:
 {'matConf': {'VP': 5, 'VN': 48, 'FP': 4, 'FN': 38}, 'recall': 0.11627906976744186, 'precisao': 0.9230769230769231}

Resultados do MLP:
 {'matConf': {'VP': 12, 'VN': 46, 'FP': 6, 'FN': 31}, 'recall': 0.27906976744186046, 'precisao': 0.8846153846153846}


Com os parâmetros de entrada acertados no item 2, podemos ver que o algoritmo do MLP se saiu melhor na predição, com as melhores taxas de recall e precisão.